In [9]:
import openai
import dotenv
import os
import requests
import yaml
import tqdm
from caseconverter import macrocase

dotenv.load_dotenv()

client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)


In [10]:
items_pdata_path = "../.projor/data/items.pdata.yaml"
with open(items_pdata_path, "r") as f:
    items = yaml.safe_load(f)['objects']

In [11]:
def capital_case(macrocasename):
    parts = macrocasename.split(" ")
    capitalized_parts = [part.capitalize() for part in parts]
    return " ".join(capitalized_parts)
def kebab_case(macrocasename):
    parts = macrocasename.split(" ")
    kebab_parts = [part.lower() for part in parts]
    return "-".join(kebab_parts)
def render_prompt(itemname):
    return f"cut-out sticker with white outline, vibrant pastel color scheme, vector graphics illustration, completely solid black background, subject is the element '{capital_case(macrocase(itemname))}' from a game"
def image_filename_of(itemname):
    return "../packages/assets/src/items/item." + kebab_case(itemname) + ".png"

In [12]:
def gen_image_with_retry(prompt, outfilename):
    for i in range(10):
        try:
            # Generate an image based on the prompt
            response = openai.images.generate(prompt=prompt, model="dall-e-3")
            response_image = response.data[0]
            response_image_url = response_image.url

            # Prints response containing a URL link to image
            image_download_response = requests.get(response_image_url)
            image_download_response.raise_for_status()
            with open(outfilename, "wb") as f:
                f.write(image_download_response.content)
            return
        except Exception as e:
            print(f"Retrying for prompt '{prompt}' and filename '{outfilename}' ...")
            continue
    print(f"Failed to generate image for prompt '{prompt}' and filename '{outfilename}'")
    raise Exception(f"Failed to generate image for prompt '{prompt}' and filename '{outfilename}'")

In [13]:
for item in tqdm.tqdm(items):
    itemname = item['name']
    prompt = render_prompt(itemname)
    outfilename = image_filename_of(itemname)
    if os.path.exists(outfilename):
        print(f"Skipping {outfilename} as it already exists")
        continue
    print(f"Generating image for {itemname} to {outfilename} ...")
    gen_image_with_retry(prompt, outfilename)
    print(f"Generated image for {itemname} to {outfilename}!")

  0%|          | 0/172 [00:00<?, ?it/s]

Generating image for Monkey to ../packages/assets/src/items/item.monkey.png ...


  0%|          | 0/172 [00:14<?, ?it/s]

Failed to generate image for prompt 'cut-out sticker with white outline, vibrant pastel color scheme, vector graphics illustration, completely solid black background, subject is the element 'Monkey' from a game' and filename '../packages/assets/src/items/item.monkey.png'


Exception: Failed to generate image for prompt 'cut-out sticker with white outline, vibrant pastel color scheme, vector graphics illustration, completely solid black background, subject is the element 'Monkey' from a game' and filename '../packages/assets/src/items/item.monkey.png'